# GoogleNet

```{note}
我们该用什么样大小的卷积？GoogleNet说我全都要！<br/>
GoogleNet是2014年ImageNet比赛的冠军
```

## 结构

在GoogleNet中，基本的卷积块被称为Inception块，它由四条并行的通路组成，各自使用了不同大小的卷积核。

前3条通路的开头和第4条通路的末尾都是用了 $1\times{1}$ 的卷积核，那它究竟起什么作用呢？事实上与 $1\times{1}$ 卷积核卷积后，output是input各channel的线性组合，权重就是此 $1\times{1}$ 的卷积核各channel。在实际使用中，它主要起改变通道数的作用。

这4条通路都使用合理的参数使得输入和输出的高和宽一致，最后在通道维度上连接。

![jupyter](../images/d/inception.svg)

In [1]:
import torch
from torch import nn
import torch.nn.functional as F


class Inception(nn.Module):
    """
    GoogleNet中的Inception块
    自定义模块，主要定义好forward
    """
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        # c1到c4表示个路中的通道数
        super(Inception, self).__init__(**kwargs)
        # 1 x 1 卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 1 x 1 卷积层 + 3 x 3 卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 1 x 1 卷积层 + 5 x 5 卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 3 x 3 max-pooling层 + 1 x 1 卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        # 计算各通路
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在channels维度上连接起来
        return torch.cat((p1, p2, p3, p4), dim=1)

GoogleNet一共使用了9个Inception块，各块之间是Pooling层。

Global AvgPool即全局池化层，将一个channel中所有点的平均值作为池化结果，即其output的高和宽均为1。

网络开头是数个卷积层，结尾是全连接层。

![jupyter](../images/d/inception-full.svg)

## 实现

In [2]:
# 第一段顺序卷积
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(), 
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [3]:
# 第二段顺序卷积
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1), nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [4]:
# 第一段Inceptions
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [5]:
# 第二段Inceptions
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [6]:
# 第三段Inceptions
# AdaptiveAvgPool2d((1, 1))的效果就是Global AvgPool
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten())

In [7]:
# GoogleNet
net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

In [8]:
X = torch.rand(size=(1, 1, 96, 96))
# 打印各部分的shape
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


## 训练

In [9]:
import d2l

# 载入数据
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size, resize=96)

In [10]:
# 虚空训练
lr, num_epochs = 0.1, 10
# d2l.train_image_classifier(net, train_iter, test_iter, lr, num_epochs)